In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor

data = load_boston()
data.keys()

X, y = data["data"], data["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

pipe = Pipeline([('std_scl',StandardScaler())])
X_train = pipe.fit_transform(X_train)
X_test = pipe.transform(X_test)

X_train = pd.DataFrame(X_train,columns=data["feature_names"])
X_test = pd.DataFrame(X_test,columns=data["feature_names"])
y_train = pd.DataFrame(y_train,columns=["target"])
y_test = pd.DataFrame(y_test,columns=["target"])

In [2]:
from sklearn.ensemble import VotingRegressor

vote_reg = VotingRegressor([('knn_reg',KNeighborsRegressor()),
                            ('Ridge_reg',Ridge()),
                            ('dt_reg',DecisionTreeRegressor(max_depth=3))
                           ],
                          weights=None,
                          n_jobs=2)
vote_reg.fit(X_train, y_train)

/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_voting.py:406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


VotingRegressor(estimators=[('knn_reg',
                             KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                                 metric='minkowski',
                                                 metric_params=None,
                                                 n_jobs=None, n_neighbors=5,
                                                 p=2, weights='uniform')),
                            ('Ridge_reg',
                             Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                                   max_iter=None, normalize=False,
                                   random_state=None, solver='auto',
                                   tol=0.001)),
                            ('dt_reg',
                             DecisionTreeRegressor(ccp_alpha=0.0,
                                                   criterion='mse', max_depth=3,
                                                   max_features=None,
                            

In [3]:
for estimator,est_name in zip(vote_reg.estimators_,vote_reg.named_estimators_):
    print(f'{est_name}:',round(estimator.score(X_test,y_test),3))
print('Voting regressor:',round(vote_reg.score(X_test,y_test),3))

knn_reg: 0.562
Ridge_reg: 0.588
dt_reg: 0.59
Voting regressor: 0.646
